In [1]:
from openalea.plantgl.all import *
from openalea.mtg import *
from oawidgets.plantgl import PlantGL

# Draw a leaf
A leaf can be a complex shape or just a simple triangle

In [2]:
def bezier_leaf():
    l = []
    l.append([Vector4(-0.00170203, 0.00487903, -0.00155362, 1),
              Vector4(-0.00946124, 0.0267487, 0.00857975, 1),
              Vector4(-0.0145598, 0.0310762, 0.0565383, 1),
              Vector4(-0.00422035, 0.0237428, 0.101953, 1)])
    l.append([Vector4(3.9604e-05, 0.0019996, 5.60056e-06, 1),
              Vector4(-0.00466331, 0.00703859, 0.0339818, 1),
              Vector4(-0.00868596, 0.00523895, 0.076457, 1),
              Vector4(0.00379859, 0.00943444, 0.154352, 1)])
    l.append([Vector4(-3.9604e-05, -0.0019996, -5.60056e-06, 1),
              Vector4(-0.00493527, 0.00263947, 0.0352765, 1),
              Vector4(-0.00867663, 0.00259947, 0.0760139, 1),
              Vector4(0.00447972, 0.00923447, 0.156651, 1)])
    l.append([Vector4(-0.00218137, -0.00475904, 0.000459084, 1),
              Vector4(-0.0120507, -0.0206578, 0.0115464, 1),
              Vector4(-0.0150292, -0.0230562, 0.0604107, 1),
              Vector4(-0.00608397, -0.0102688, 0.102558, 1)])
    
    matrix = Point4Matrix(4, 4)
    for i, row in enumerate(l):
        for j, pt in enumerate(row):
            pt = pt * 9
            pt.w = 1
            matrix[i, j] = pt
    
    leaf = BezierPatch(matrix, 4, 4)

    leaf.name = 'leaf'
    return leaf


In [3]:
leaf=bezier_leaf()

In [4]:
PlantGL(leaf)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [5]:
# list of points
points =  [(-1,-1,0),
           (1,-1,0),
           (1,1,0),
           (-1,1,0)]
# list of indices
indices = [(0, 1, 2),(2,3,0)]
# creating the geometry
simple_leaf = TriangleSet(pointList=points, indexList=indices)
PlantGL(simple_leaf)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

# Generate a tree (Cylinder + list of leaves)

In [6]:
tree = []

def trunk(height=30):
    return Cylinder(radius=1, height=height)

PlantGL(trunk())

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [8]:
from random import uniform
from math import cos, sin, radians, pi

def spherical(radius):
    theta = uniform(-pi, pi)
    phi = uniform(-pi/2, pi/2)
    return [radius * cos(theta) * cos(phi),
            radius * sin(theta) * cos(phi),
            radius *sin(phi)]
            
def leaves(nb_leaves=1000, height=30, radius=20):
    l = []
    
    rads = [0, 5, 15, 20]
    p1, p2, p3 = 0.1, 0.6, 0.3
    
    for i in range(nb_leaves):
        p = uniform(0,1)
        if p <= p1:
            rad = uniform(rads[0], rads[1])
        elif p1< p <=p2:
            rad = uniform(rads[1], rads[2])
        else:
            rad = uniform(rads[2], rads[3])

        position = spherical(rad)
        position[2] +=(height+radius)

        az, el, roll = uniform(-pi, pi), uniform(0, pi), uniform(-pi, pi)
        _leaf = Translated(position, EulerRotated (az, el, roll, simple_leaf))

        l.append(_leaf)

    return l

scene = Scene(leaves())
scene+=trunk()
PlantGL(scene)
        
        

            

    
    

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…